In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/Shahwar-Masters-Project-Protein-protein-interactions/Shahwar-Code

/content/drive/MyDrive/Shahwar-Masters-Project-Protein-protein-interactions/Shahwar-Code


In [3]:
!pip install pickle5

     |████████████████████████████████| 256 kB 5.2 MB/s 


In [4]:
from keras.layers import Conv1D, MaxPooling1D, Embedding,  Concatenate, Lambda
from keras.models import Model
from sklearn.metrics import roc_auc_score,roc_curve, auc
from numpy import random
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout
from tensorflow.keras.optimizers import Adam
from keras.utils.vis_utils import plot_model
import seaborn as sns
import functions as f
import pandas as pd
import pickle5 as pickle
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, GlobalMaxPooling1D, Flatten

In [6]:
lengths = []
for i in range(df_test.shape[0]):
  lengths.append(len(df_test[['Joined']].iloc[i][0]))

s = sorted(lengths)
print("median test Joined sequences length:", s[len(s) // 2])

small_seq = df_test.loc[[i for i, x in enumerate(lengths) if x < 500]].reset_index(drop=True)
y_small_seq = df_test['label'].loc[[i for i, x in enumerate(lengths) if x < 500]].reset_index(drop=True)
big_seq = df_test.loc[[i for i, x in enumerate(lengths) if x > 1500]].reset_index(drop=True)
y_big_seq = df_test['label'].loc[[i for i, x in enumerate(lengths) if x > 1500]].reset_index(drop=True)
small_seq.shape,big_seq.shape

NameError: ignored

In [7]:
name = 'model_final_4_0.919'
model = load_model('/content/drive/MyDrive/ML_Data/final_mode

SyntaxError: ignored

In [10]:
# df_train,df_test = load_data(5)
# print('The data used will be:')
# df_train
def load_data(D,randomize=False):
    try:
        with open("df_train_"+str(D)+"D.pkl", "rb") as fh:
          df_train = pickle.load(fh)
    except:
        df_train = pd.read_pickle("hyppi-train-data.pkl")
    try:
        with open("df_test_"+str(D)+"D.pkl", "rb") as fh:
          df_test = pickle.load(fh)
    except:
        df_test = pd.read_pickle("hyppi-test-data.pkl")
    if randomize:
        return f.shuff_together(df_train,df_test)
    else:
        return df_train,df_test

df_train,df_test = load_data(1)
print(df_train)


                                                  Human  \
0     [M, S, V, E, A, Y, G, P, S, S, Q, T, L, T, F, ...   
1     [M, V, P, G, E, E, N, Q, L, V, P, K, E, D, V, ...   
2     [M, T, A, T, V, E, N, L, T, F, Q, K, D, T, L, ...   
3     [M, S, A, L, C, W, G, R, G, A, A, G, L, K, R, ...   
4     [M, S, V, P, S, A, L, M, K, Q, P, P, I, Q, S, ...   
...                                                 ...   
2995  [M, P, A, R, P, G, R, L, L, P, L, L, A, R, P, ...   
2996  [M, A, A, T, K, R, K, R, R, G, G, F, A, V, Q, ...   
2997  [M, A, A, A, A, A, V, G, N, A, V, P, C, G, A, ...   
2998  [M, P, A, R, R, L, L, L, L, L, T, L, L, L, P, ...   
2999  [M, S, G, E, V, R, L, R, Q, L, E, Q, F, I, L, ...   

                                               Pathogen  \
0     [M, K, N, P, K, E, E, I, R, R, I, R, I, V, N, ...   
1     [G, G, G, T, G, E, T, L, G, E, K, W, K, A, R, ...   
2     [G, G, G, T, G, E, T, L, G, E, K, W, K, A, R, ...   
3     [M, K, N, P, K, E, E, I, R, R, I, R, I, V, N, ...

In [11]:
df_test

,Human,Pathogen,Joined,label
0,"[M, G, I, S, R, D, N, W, H, K, R, R, K, T, G, ...","[M, K, N, P, K, K, K, S, G, G, F, R, I, V, N, ...","[M, G, I, S, R, D, N, W, H, K, R, R, K, T, G, ...",1
1,"[M, V, G, E, E, K, M, S, L, R, N, R, L, S, K, ...","[S, W, P, P, S, E, V, L, T, A, V, G, L, I, C, ...","[M, V, G, E, E, K, M, S, L, R, N, R, L, S, K, ...",0
2,"[M, R, R, A, G, L, G, E, G, V, P, P, G, N, Y, ...","[M, K, N, P, K, E, E, I, R, R, I, R, I, V, N, ...","[M, R, R, A, G, L, G, E, G, V, P, P, G, N, Y, ...",0
3,"[M, S, V, P, S, A, L, M, K, Q, P, P, I, Q, S, ...","[G, G, G, T, G, E, T, L, G, E, K, W, K, A, R, ...","[M, S, V, P, S, A, L, M, K, Q, P, P, I, Q, S, ...",1
4,"[M, A, P, V, E, H, V, V, A, D, A, G, A, F, L, ...","[M, K, N, P, K, E, E, I, R, R, I, R, I, V, N, ...","[M, A, P, V, E, H, V, V, A, D, A, G, A, F, L, ...",0
...,...,...,...,...
574,"[M, N, H, L, P, E, D, M, E, N, A, L, T, G, S, ...","[G, A, A, F, G, V, M, E, A, L, G, T, L, P, G, ...","[M, N, H, L, P, E, D, M, E, N, A, L, T, G, S, ...",0
575,"[M, S, F, R, G, G, G, R, G, G, F, N, R, G, G, ...","[M, K, N, P, K, E, E, I, R, R, I, R, I, V, N, ...","[M, S, F, R, G, G, G, R, G, G, F, N, R, G, G, ...",0
576,"[M, T, K, N, E, K, K, S, L, N, Q, S, L, A, E, ...","[M, K, N, P, K, E, E, I, R, R, I, R, I, V, N, ...","[M, T, K, N, E, K, K, S, L, N, Q, S, L, A, E, ...",0
577,"[M, A, G, F, G, A, M, E, K, F, L, V, E, Y, K, ...","[N, E, L, G, W, L, E, R, T, K, N, D, I, A, H, ...","[M, A, G, F, G, A, M, E, K, F, L, V, E, Y, K, ...",1


In [ ]:
trains = f.preprocess(df_train)
tests = f.preprocess(df_test)

Converting to 5D. This will take a few minutes
Create tokenizers? Enter y if this is new training data. y/n: y
Saved tokenizers as doubleip_tkrs
Saved tokenizer as join_tkr
Preprocessing...
sentences are
Converting to 5D. This will take a few minutes
Create tokenizers? Enter y if this is new training data. y/n: n
Preprocessing...
sentences are


In [ ]:
EMBEDDING_DIM_5D = 25
VALIDATION_SPLIT = 0.2
BATCH_SIZE = 32
EPOCHS = 1
DROP = 0.5
threshold = 0.918
MAX_SEQUENCE_LENGTH_5D_J = 2000
MAX_SEQUENCE_LENGTH_5D_dIP = 1000
num_words_5D_J = 10000000
num_words_5D = 5000000

for i in range(10):


  x1_join = f.transf_model(MAX_SEQUENCE_LENGTH_5D_J,num_words_5D_J,5,0.9)
  x2_join = f.transf_model(MAX_SEQUENCE_LENGTH_5D_J,num_words_5D_J,5,0.9)
  x3_join = f.transf_model(MAX_SEQUENCE_LENGTH_5D_J,num_words_5D_J,5,0.9)

  x1_doubleip = f.conv_model(MAX_SEQUENCE_LENGTH_5D_dIP,EMBEDDING_DIM_5D,num_words_5D,DROP)
  x2_doubleip = f.conv_model(MAX_SEQUENCE_LENGTH_5D_dIP,EMBEDDING_DIM_5D,num_words_5D,DROP)
  x3_doubleip = f.conv_model(MAX_SEQUENCE_LENGTH_5D_dIP,EMBEDDING_DIM_5D,num_words_5D,DROP)
  x4_doubleip = f.conv_model(MAX_SEQUENCE_LENGTH_5D_dIP,EMBEDDING_DIM_5D,num_words_5D,DROP)
  x5_doubleip = f.conv_model(MAX_SEQUENCE_LENGTH_5D_dIP,EMBEDDING_DIM_5D,num_words_5D,DROP)
  x6_doubleip = f.conv_model(MAX_SEQUENCE_LENGTH_5D_dIP,EMBEDDING_DIM_5D,num_words_5D,DROP)

  concatenator = Concatenate(axis=1)
  x = concatenator([x1_join.output, x2_join.output, x3_join.output, x1_doubleip.output, x2_doubleip.output, x3_doubleip.output, x4_doubleip.output, x5_doubleip.output, x6_doubleip.output])
  x = Dense(256)(x)
  x = Dropout(0.5)(x)
  output = Dense(1, activation="sigmoid",name="Final")(x)
  model_final = Model(inputs=[x1_join.input, x2_join.input, x3_join.input, x1_doubleip.input, x2_doubleip.input, x3_doubleip.input, x4_doubleip.input, x5_doubleip.input, x6_doubleip.input], outputs=output)

  model_final.compile(loss='binary_crossentropy', optimizer='adam')

  model_final.fit(trains, df_train['label'].values, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(tests,df_test['label'].values))
  score = roc_auc_score(df_test['label'].values, model_final.predict(tests))
  score = round(score,3)
  print("Score is",score)
  if score>threshold:
    threshold = score
    print('Saving model_final_'+str(score)+'')
    model_final.save('model_final_'+str(i)+'_'+str(score)+'')
  

94/94 [==============================] - 327s 3s/step - loss: 0.7499 - val_loss: 0.6968
Score is 0.496
38/94 [===========>..................] - ETA: 6:20 - loss: 0.7858